In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 28s - loss: 2.7697 - auc: 0.5035 - accuracy: 0.5033 - val_loss: 0.6303 - val_auc: 0.7764 - val_accuracy: 0.7917
Epoch 2/30
40/40 - 27s - loss: 0.8900 - auc: 0.5444 - accuracy: 0.5329 - val_loss: 1.2536 - val_auc: 0.7942 - val_accuracy: 0.1901
Epoch 3/30
40/40 - 27s - loss: 0.8475 - auc: 0.5988 - accuracy: 0.5640 - val_loss: 2.2244 - val_auc: 0.8269 - val_accuracy: 0.1745
Epoch 4/30
40/40 - 27s - loss: 1.0312 - auc: 0.5920 - accuracy: 0.5624 - val_loss: 0.3677 - val_auc: 0.8303 - val_accuracy: 0.8333
Epoch 5/30
40/40 - 27s - loss: 0.9399 - auc: 0.6134 - accuracy: 0.5790 - val_loss: 1.3653 - val_auc: 0.8463 - val_accuracy: 0.2526
Epoch 6/30
40/40 - 26s - loss: 0.9208 - auc: 0.6402 - accuracy: 0.5845 - val_loss: 0.5682 - val_auc: 0.8624 - val_accuracy: 0.6484
Epoch 7/30
40/40 - 27s - loss: 0.7648 - auc: 0.6781 - accuracy: 0.6227 - val_loss: 0.7938 - val_auc: 0.8786 - val_accuracy: 0.5026
Epoch 8/30
40/40 - 27s - loss: 0.5987 - auc: 0.7596 - accuracy: 0.6920 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

359
27


In [8]:
prediction

array([[0.94704777],
       [0.9949463 ],
       [0.47331408],
       [0.7716153 ],
       [0.16384543],
       [0.6214186 ],
       [0.456604  ],
       [0.9359194 ],
       [0.99917775],
       [0.58595264],
       [0.96102947],
       [0.82102484],
       [0.8480568 ],
       [0.99582064],
       [0.83905125],
       [0.9890456 ],
       [0.9899873 ],
       [0.99707067],
       [0.98351777],
       [0.53848946],
       [0.7893765 ],
       [0.81160617],
       [0.7977972 ],
       [0.98402923],
       [0.8637631 ],
       [0.4609705 ],
       [0.4825091 ],
       [0.9578148 ],
       [0.897711  ],
       [0.9877594 ],
       [0.9319193 ],
       [0.6931194 ],
       [0.93817484],
       [0.26949155],
       [0.83281934],
       [0.9829317 ],
       [0.8622081 ],
       [0.91405237],
       [0.9348374 ],
       [0.87457824],
       [0.97227263],
       [0.89844245],
       [0.92855424],
       [0.9782628 ],
       [0.94708645],
       [0.89507145],
       [0.90664756],
       [0.355

In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

7/7 [==============================] - 3s 408ms/step - loss: 0.2410 - auc: 0.9487 - accuracy: 0.8782


[0.24095118045806885, 0.9487215876579285, 0.878238320350647]